In [ ]:
%%capture
!pip install RISE
!pip install --upgrade google-api-core
!pip install --upgrade google-api-python-client
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade grpcio

import IPython
IPython.Application.instance().kernel.do_shutdown(True)

# Advanced Python

 ![Python](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/600px-Python-logo-notext.svg.png)

## Rich Conboy & Kevin Kelleher

 ![Python](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)

# What this class is
 - Tips and tricks to improve your experience working with Python at The Home Depot
 - Strategies for speeding up workflows you may already be using
 
# What this class is not
 - Intro to programming/python
 - Data mining / machine learning tutorials (go see Matt Morton's class!)

# Agenda
- JupyterHub Basics
- Faster downloads with Pandas + BigQuery Storage API
- Python Productivity
- UI Design with ipywidgets

# What the heck is JupyterHub and why do I care?
![Logo](https://www.dataquest.io/wp-content/uploads/2019/01/1-LPnY8nOLg4S6_TG0DEXwsg-1.png)

**Jupyter Notebook** is a web application that allows the end-user to create documents that contain live code (in python, R, or Julia), equations, visualizations and narrative Markup Markdown text.  Jupyter Notebooks are used for data cleansing, data transformation, statistical modeling, data visualization, and machine learning.

![Jupyter Notebook](https://www.dataschool.io/content/images/2019/03/binder-50.png)

**JupyterHub** is a multi-user server that manages and proxies multiple instances of a single-user of JupyterLab or Jupyter Notebooks.  JupyterHub is the best way to serve Jupyter notebook for multiple users. It can be used in a classes of students, a corporate data science group or scientific research group.  It is a multi-user Hub that spawns, manages, and proxies multiple instances of the single-user Jupyter notebook server.

![JupyterHub Architecture](https://jupyterhub.readthedocs.io/en/stable/_images/jhub-fluxogram.jpeg)

# Using JupyterHub

### Getting Access:
- **Required:** GG_CLOUD_GCP_your-cloud-project-id_DATA_ANALYST or GG_CLOUD_GCP_your-cloud-project-id_DATA_SCIENTIST (to get read / write access to your team's business project)
- **Required:** GG_CLOUD_EDW_DATA_USER (to get read access to the data in the IT supported pr- projects)
- **Required:** GG_CLOUD_GCP_your-cloud-project-id_DATALAB_USER (to get access to JupyterHub)
- **Optional:** GG_CLOUD_GCP_your-cloud-project-id_DATALAB_POWERUSER in addition to GG_CLOUD_GCP_your-cloud-project-id_DATALAB_USER (to spawn medium or large size servers)

# Using JupyterHub

### Pricing:
Once you spawn a server, there is a charge for the first minute and then every second after that.  **Best practice is to use the smallest instance possible for your use case.**  Start with the small server and move up if necessary.

| Spawner Selection   | Cores and RAM                | Cost               |
| :------------------ | :--------------------------: | :----------------: |
| Small Instance      | 2 cores up to 13 gb of RAM   | \$0.19 per hour    |
| Medium Instance     | 8 cores up to 52 gb of RAM   | \$0.76 per hour    |
| Large Instance      | 64 cores up to 416 gb of RAM | \$4.56 per hour    |

# Using JupyterHub

### When to Use What?

|                                                                             | JupyterHub    | Local Workstation |
| :-------------------------------------------------------------------------- | :-----------: | :---------------: |
| Flexible computing power and memory options (up to 64 cores & 416 GB RAM)   | &#10004;      |                   |
| No fiddling with Google Cloud SDK, gsutil, or security certificates         | &#10004;      |                   |
| Super fast download to Pandas DataFrame via Cloud Storage API               | &#10004;      |                   |
| Package management directly in notebooks preserves compatibility            | &#10004;      |                   |
| Free                                                                        |               | &#10004;          |
| Python virtual environment flexibility via Anaconda                         |               | &#10004;          |
| Zero downtime or maintenance issues                                         |               | &#10004;          |
| Interface with on-prem data (SQL Server anyone?) and local tools            |               | &#10004;          |

# Fast download to Pandas via Cloud Storage API

One of the most common things we find ourselves doing anytime we are working with data in Python is downloading it from BigQuery.  Historically, this is probably one of the biggest pain points we've seen in the Home Depot environment.  I'll bet most of the people here have run some flavor of the code below:

In [ ]:
import pandas as pd

sql = """
    SELECT *
    FROM `analytics-supplychain-thd.Rich.ALLOC_HIST`
    LIMIT 100000
"""

# Run a Standard SQL query with the project set explicitly
df = pd.read_gbq(sql, project_id='analytics-supplychain-thd', dialect='standard')

### We can do better by adding a couple of optional components of google-cloud-bigquery...

In [ ]:
%%capture
!pip install --upgrade google-cloud-bigquery[bqstorage,pandas,pyarrow]

import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import pandas as pd

sql = """
    SELECT *
    FROM `analytics-supplychain-thd.Rich.ALLOC_HIST`
    LIMIT 100000
"""

# Run a Standard SQL query with the project set explicitly
df = pd.read_gbq(sql, project_id='analytics-supplychain-thd', dialect='standard')

# Python Productivity
<p>In this section, we'll walk through an example problem and show some tips and tricks along the way to make you more productive and make your code faster and more accurate.</p>
<p>The problem: we have a list of POs ranked by priority, and we have to choose which POs to include on the next truck while making sure the truck is as close to 2,400 cube as possible. We've also just been given a list of priority SKUs, so first we need to update the existing priority of the POs if it contains any of the priority SKUs.</p>
<p>First, we use pandas to pull in our data and print out the first five rows:</p>

In [ ]:
import pandas as pd

sku_df = pd.read_gbq(
'''
SELECT * FROM `analytics-supplychain-thd.Kevin.SOD_SKUS`
'''
    , dialect='standard'
    , project_id='analytics-supplychain-thd'
)

po_df = pd.read_gbq(
'''
SELECT * FROM `analytics-supplychain-thd.Kevin.SOD_POS`
'''
    , dialect='standard'
    , project_id='analytics-supplychain-thd'
)

In [ ]:
sku_df.head()

In [ ]:
po_df.head()

<p>Below is our first attempt at updating the PO priority based on our priority SKU list. We use the pandas apply function to call our update_priority function on each row. We loop through each priority SKU, and check if it's included on the PO. If it is, we add 50,000 to the existing priority score. Otherwise, we return the original priority. Let's test it out on the first 1,000 POs:</p>

In [ ]:
%%time

priority_skus = list(sku_df['SKU_NBR'])

def update_priority(row):
    for priority_sku in priority_skus:
        if str(priority_sku) in row['SKUS']:
            return row['PRIORITY'] + 50000
    return row['PRIORITY']
    
po_df.head(1000).apply(update_priority, axis=1)

<p>23 seconds for only 1,000 rows?? At this rate, it will take an hour to update all of our data!</p>
<p>Our first takeaway here should be to <strong>use SQL whenever possible</strong> - SQL joins are very optimized and prevent exactly the issue we have here.</p>
<p>Sometimes that's not an option though, so let's see if we can speed up our code. An easy option is to use the %%prun magic, which creates a profile of which operations in your code are taking the longest.</p>
<p>In our case, the profile isn't very helpful, but if your code is more complex than our example here, it can help point you in the right direction.</p>

In [ ]:
%%prun

priority_skus = list(sku_df['SKU_NBR'])

def update_priority(row):
    for priority_sku in priority_skus:
        if str(priority_sku) in row['SKUS']:
            return row['PRIORITY'] + 50000
    return row['PRIORITY']
    
po_df.head(1000).apply(update_priority, axis=1)

<p>If we think about our code, we can see that for each row in po_df (~150K rows), we evaluate each priority SKU (~4K SKUs), which means we're attempting to perform ~600 million operations - no wonder the code is slow!</p>
<p>If we google something like "python fastest way to check if element in list," we may get the idea to try using sets rather than lists. Let's try it out!</p>

In [ ]:
priority_skus = list(sku_df['SKU_NBR'])

priority_skus_set = set(sku_df['SKU_NBR'])

In [ ]:
%%timeit
1 in priority_skus

In [ ]:
%%timeit
1 in priority_skus_set

<p>After doing some googling on what exactly these units are, you'll see that using the set provides a 1,000x speedup!</p>
<p>To check if an item is in a list, we have to iterate through each element in this list.</p>
<p>On the other hand, you can think of the set as having an index - we can immediately determine whether or not the element is in the set. Python dictionaries also have this property.</p>
<p>Let's try rewriting our original code to use sets:</p>

In [ ]:
%%time

priority_skus_set = set(str(x) for x in list(sku_df['SKU_NBR']))

def update_priority(row):
    skus = set(row['SKUS'].split(','))
    if not skus.isdisjoint(priority_skus_set):
        return row['PRIORITY'] + 50000
    return row['PRIORITY']
    
po_df.apply(update_priority, axis=1)

<p>We just ran our code for every row (not just the first 1,000) in 3 seconds - again, a 1,000x speedup!</p>
<p>We forgot to save our results, but there is a trick that will prevent you from having to rerun your code. Jupyter notebook automatically saves the results from the last cell run in a variable named "_" - we can assign this as a column in our data frame.</p>

In [ ]:
po_df['NEW_PRIORITY'] = _

In [ ]:
po_df

<p>Now that our PO priorities are updated, we can return to the original problem: for each store, send the highest priority POs on the next truck while filling the truck as close to 2,400 cube as possible.</p>
<p>Below is our first attempt - we use pandas groupby-apply to separate our data for each store and apply the build_truck function separately. We sort the data by highest priority, then loop through the POs. For each PO we encounter, we check if adding it will put us over 2,400 cube - if it won't, we add it, otherwise we move on to the next PO:</p>

In [ ]:
%%time

po_df['INCLUDED'] = False

def build_truck(g):
    included_cube = 0
    g.sort_values('NEW_PRIORITY', ascending=False, inplace=True)
    for i, row in g.iterrows():
        if included_cube <= 2400:
            g.loc[i, 'INCLUDED'] = True
            included_cube += row['TTL_CUBE']
    return g

new_po_df = po_df.groupby('STR_NBR', as_index=False).apply(build_truck)

<p>No major performance issues here! Let's check our results. Below is an assert statement that checks to see that the included POs for each store total to less than 2,400 cube. The assert statement will throw an error if our test returns False.</p>

In [ ]:
assert (new_po_df[new_po_df['INCLUDED']].groupby('STR_NBR')['TTL_CUBE'].sum() <= 2400).all()

<p>That's not good! Good thing we checked our logic before sending out the results.</p>
<p>We can add an assert statement to our original code and use the %debug magic to get a better sense of what's going on - this is a really powerful combination that lets you see exactly what's going on inside your functions. The %debug magic lets us step through the code and print all of our variables at the time of the last error - much faster than adding print statements one by one.</p>

In [ ]:
po_df['INCLUDED'] = False

def build_truck(g):
    included_cube = 0
    g.sort_values('NEW_PRIORITY', ascending=False, inplace=True)
    for i, row in g.iterrows():
        if included_cube <= 2400:
            g.loc[i, 'INCLUDED'] = True
            included_cube += row['TTL_CUBE']
        assert included_cube <= 2400
    return g

new_po_df = po_df.groupby('STR_NBR', as_index=False).apply(build_truck)

Check out the debugger commands <a href="https://docs.python.org/3/library/pdb.html#debugger-commands">here.</a><br>
These take some getting used to, but are well worth taking the time to learn.

In [ ]:
%debug

<p>Ah, we're only checking if included cube is under 2,400 before adding a PO, not checking if adding the PO will push us over 2,400. Let's update the code and make sure our assert statements don't throw any errors after doing so:</p>

In [ ]:
po_df['INCLUDED'] = False

def build_truck(g):
    included_cube = 0
    g.sort_values('NEW_PRIORITY', ascending=False, inplace=True)
    for i, row in g.iterrows():
        if included_cube + row['TTL_CUBE'] <= 2400:
            g.loc[i, 'INCLUDED'] = True
            included_cube += row['TTL_CUBE']
        assert included_cube <= 2400
    return g

new_po_df = po_df.groupby('STR_NBR', as_index=False).apply(build_truck)

assert (new_po_df[new_po_df['INCLUDED']].groupby('STR_NBR')['TTL_CUBE'].sum() <= 2400).all()

<p>Now we're done! And if we have to perform this analysis again or on a regular basis, our assert statements will give us added confidence that our code continues to give correct results.</p>

# UI Design with ipywidgets

**ipywidgets** is a neat module that let's you construct mini-applications using Jupyter notebook as a front-end.

### Here's a simple example:

In [ ]:
import ipywidgets as widgets

# Create Widgets
name = widgets.Text(description='Name: ')
print_name = widgets.Button(description='Click me!')

# Define Button Action
print_name.on_click(lambda _: print('Hello {}!'.format(name.value)))

# Display UI
display(name)
display(print_name)

### A more complicated example using conditional logic...

In [ ]:
# Create Widgets
ds_toggle = widgets.Checkbox(value=False, description='Please check if you work in Supply Chain', disabled=False, indent=False)
ds_select = widgets.Dropdown(options=['Kevin', 'Samantha', 'Megan', 'Rich', 'Obi'], description='Who is the best Data Science Team Member: ', disabled=False, indent=False, layout=widgets.Layout(width='100%'), style={'description_width': 'initial'})

# Display First Widget
display(ds_toggle)

# Create Output For Interactive Widgets
ds_output = widgets.Output()
display(ds_output)

# Code to Print the Best
def on_select_change(change):
    ds_output.clear_output()
    with ds_output:
        display(ds_select)
        if ds_select.value != 'Rich':
            print(ds_select.value, 'is a good choice!')
        else:
            print('Please select again')

# Code to Make Confidence Level Show Up When Mahalanobis Toggle is Checked
def on_toggle_change(change):
    if ds_toggle.value == True:
        with ds_output:
            display(ds_select)
    else:
        ds_output.clear_output()
        
ds_select.observe(on_select_change, names='value')
ds_toggle.observe(on_toggle_change, names='value')
